In [1]:
import pandas as pd

In [2]:
# Load the dataset
df = pd.read_csv('combined_data.csv')

C:\Users\nikhi\AppData\Local\Temp\ipykernel_4760\1638853474.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('combined_data.csv')


In [3]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11710080 entries, 0 to 11710079
Data columns (total 3 columns):
 #   Column        Dtype  
---  ------        -----  
 0   time          object 
 1   xray_flux     float64
 2   flare_status  object 
dtypes: float64(1), object(2)
memory usage: 268.0+ MB
None


In [4]:
# Handle missing values in xray_flux using forward fill
df['xray_flux'] = df['xray_flux'].fillna(method='ffill')

C:\Users\nikhi\AppData\Local\Temp\ipykernel_4760\2480032580.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['xray_flux'] = df['xray_flux'].fillna(method='ffill')


In [5]:
# Double-check for any remaining NaNs and fill them
df['xray_flux'] = df['xray_flux'].fillna(method='bfill')
df = df.dropna(subset=['flare_status'])  # Drop rows where flare_status is NaN

C:\Users\nikhi\AppData\Local\Temp\ipykernel_4760\3762337873.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['xray_flux'] = df['xray_flux'].fillna(method='bfill')


In [6]:
# Verify that no NaNs are left
print(df.isna().sum())

time            0
xray_flux       0
flare_status    0
dtype: int64


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [8]:
# Define features and target
X = df[['xray_flux']]
y = df['flare_status']

In [9]:
# Normalize the xray_flux values
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [10]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Data preparation done. Ready for training!")

Data preparation done. Ready for training!


In [11]:
from sklearn.ensemble import RandomForestClassifier

In [12]:
# Modify the model to train faster
model = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=42)
model.fit(X_train, y_train)


RandomForestClassifier(max_depth=10, n_estimators=50, random_state=42)

In [13]:
# Make predictions on the test set
y_pred = model.predict(X_test)

print("Predictions made.")


Predictions made.


In [14]:
from sklearn.metrics import classification_report, accuracy_score

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Print the results
print("Model Accuracy:", accuracy)
print("\nClassification Report:\n", report)


c:\Users\nikhi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nikhi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Model Accuracy: 0.5999455110983297

Classification Report:
               precision    recall  f1-score   support

   DECLINING       0.55      0.33      0.41     52617
   NO_STATUS       0.00      0.00      0.00       329
        PEAK       0.00      0.00      0.00      4892
      RISING       0.62      0.83      0.71     74299

    accuracy                           0.60    132137
   macro avg       0.29      0.29      0.28    132137
weighted avg       0.57      0.60      0.56    132137



c:\Users\nikhi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [15]:
from imblearn.over_sampling import SMOTE

# Initialize SMOTE
smote = SMOTE(random_state=42)

# Resample the training data
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Check the new class distribution
print(pd.Series(y_train_resampled).value_counts())


flare_status
DECLINING    297657
RISING       297657
PEAK         297657
NO_STATUS    297657
Name: count, dtype: int64


In [16]:
# Train the Random Forest model with the resampled data
model.fit(X_train_resampled, y_train_resampled)


RandomForestClassifier(max_depth=10, n_estimators=50, random_state=42)

In [17]:
# Make predictions on the test set
y_pred_resampled = model.predict(X_test)

# Evaluate the model's performance
accuracy_resampled = accuracy_score(y_test, y_pred_resampled)
report_resampled = classification_report(y_test, y_pred_resampled)

print("Model Accuracy after Resampling:", accuracy_resampled)
print("\nClassification Report after Resampling:\n", report_resampled)


Model Accuracy after Resampling: 0.24648660102772124

Classification Report after Resampling:
               precision    recall  f1-score   support

   DECLINING       0.53      0.16      0.24     52617
   NO_STATUS       0.00      0.44      0.01       329
        PEAK       0.05      0.52      0.10      4892
      RISING       0.89      0.29      0.44     74299

    accuracy                           0.25    132137
   macro avg       0.37      0.35      0.20    132137
weighted avg       0.71      0.25      0.35    132137



In [18]:
# Modify the model to include class_weight balancing
model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')

# Train the model
model.fit(X_train, y_train)



RandomForestClassifier(class_weight='balanced', random_state=42)

In [19]:
# Make predictions on the test set
y_pred_balanced = model.predict(X_test)

# Evaluate the model's performance
accuracy_balanced = accuracy_score(y_test, y_pred_balanced)
report_balanced = classification_report(y_test, y_pred_balanced)

print("Model Accuracy with Class Weight Balancing:", accuracy_balanced)
print("\nClassification Report with Class Weight Balancing:\n", report_balanced)


Model Accuracy with Class Weight Balancing: 0.5540083398291168

Classification Report with Class Weight Balancing:
               precision    recall  f1-score   support

   DECLINING       0.49      0.49      0.49     52617
   NO_STATUS       0.00      0.01      0.01       329
        PEAK       0.05      0.06      0.05      4892
      RISING       0.64      0.63      0.64     74299

    accuracy                           0.55    132137
   macro avg       0.30      0.30      0.30    132137
weighted avg       0.56      0.55      0.56    132137



In [20]:
df['hour'] = pd.to_datetime(df['time']).dt.hour
df['day'] = pd.to_datetime(df['time']).dt.day
df['month'] = pd.to_datetime(df['time']).dt.month


In [21]:
df['xray_flux_rolling_mean'] = df['xray_flux'].rolling(window=5).mean().shift(1)


In [22]:
df['xray_flux_lag1'] = df['xray_flux'].shift(1)


In [23]:
df = df.dropna()  # Drop rows with NaNs generated by shifting


In [24]:
X = df.drop(['flare_status', 'time'], axis=1)
y = df['flare_status']

In [25]:
# Train the model
model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', random_state=42)

In [26]:

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Model Accuracy with Feature Engineering:", accuracy)
print("\nClassification Report with Feature Engineering:\n", report)

Model Accuracy with Feature Engineering: 0.5540083398291168

Classification Report with Feature Engineering:
               precision    recall  f1-score   support

   DECLINING       0.49      0.49      0.49     52617
   NO_STATUS       0.00      0.01      0.01       329
        PEAK       0.05      0.06      0.05      4892
      RISING       0.64      0.63      0.64     74299

    accuracy                           0.55    132137
   macro avg       0.30      0.30      0.30    132137
weighted avg       0.56      0.55      0.56    132137



In [28]:
X = df[['time', 'xray_flux']]
y = df['flare_status']

In [29]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [31]:
import joblib

In [32]:
# Save the trained model
joblib.dump(model, 'solar_flare_model.pkl')

# Save the scaler (since you'll need it for prediction)
joblib.dump(scaler, 'scaler.pkl')


['scaler.pkl']

In [33]:
# Load the model
model = joblib.load('solar_flare_model.pkl')

# Load the scaler
scaler = joblib.load('scaler.pkl')
